

# **IFI8410: Programming for Business**



## **Assignment 07**

In [ ]:
# Do not change the content of this cell. Execute this cell first, and everytime after you restarted the kernel.
%reload_ext autoreload
%autoreload 2

### 7.1. A Brief Matplotlib API Primer

Question:

You are given a CSV file **sample_sales_data_new.csv** containing the following columns:

- Region: The region where the sale occurred.
- Salesperson: The name of the salesperson.
- Product: The product that was sold.
- Quantity: The quantity of the product sold.
- SalesAmount: The total sales amount for the sale.
- Write a function that takes a DataFrame and groups the data by Region and Product, returning the total Quantity sold and the total SalesAmount for each combination.

Hint:

- Use the `groupby()` function to group the data by Region and Product.
- Use the `agg()` function to apply aggregation operations (sum) on Quantity and SalesAmount.

In [ ]:
import pandas as pd

def group_and_aggregate_sales(df: pd.DataFrame) -> pd.DataFrame:
    
    # Write your code here
    


In [ ]:
#EXAMPLE USAGE
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

grouped_sales = group_and_aggregate_sales(df)

print(grouped_sales)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_1.py

import pandas as pd

def group_and_aggregate_sales(df: pd.DataFrame) -> pd.DataFrame:



#### Test 7.1 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 1

### 7.2 Iterating over Groups

Question:

Using **samples_sales_data.csv** write a function that iterates over the groups in the DataFrame grouped by Region and returns a dictionary where each key is a region and the value is the DataFrame corresponding to that region.

Hint:

- Use the `groupby()` function to group the data by Region.
- Iterate over the grouped object using a for loop to get both the group name and the DataFrame.


In [ ]:
import pandas as pd

def iterate_over_groups(df: pd.DataFrame) -> dict:
    
    # Write your code here
    


In [ ]:
# Example Usage
# Load the CSV file into a DataFrame
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

# Call the function
grouped_sales = iterate_over_groups(df)

#Print the result for each group
for region, group_df in grouped_sales.items():
    print(f"\nRegion: {region}")
    print(group_df)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_2.py

import pandas as pd

def iterate_over_groups(df: pd.DataFrame) -> dict:



#### Test 7.2 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 2

### 7.3 Grouping with Dictionaries and Series

You are provided with a region mapping dictionary that groups several regions into broader zones. 
Your task is to group the data by these zones and calculate the total **Quantity** sold and the total **SalesAmount** for each zone.

- region_mapping = {
    'North': 'Zone 1',
    'South': 'Zone 2',
    'East': 'Zone 1',
    'West': 'Zone 2'
}

Question:

Write a function that takes a DataFrame and a region mapping dictionary as inputs, replaces the Region column values with the corresponding zone, and returns the aggregated **Quantity** and **SalesAmount** for each zone.

Let the solution function return a DataFrame with columns **Zone**, **Quantity**, **SalesAmount** with **Quantity**, **SalesAmount** containing the aggregated totals for each region.

Hint:

- Use `replace()` to map the regions to zones.
- Use `groupby()` to group by the new zone column.

In [ ]:
import pandas as pd

def group_by_zone(df: pd.DataFrame, region_mapping: dict) -> pd.DataFrame:
    
    # Write your code here
    


In [ ]:
# Example Usage

df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

region_mapping = {
    'North': 'Zone 1',
    'South': 'Zone 2',
    'East': 'Zone 1',
    'West': 'Zone 2'
}

# Call the function
zone_sales = group_by_zone(df, region_mapping)

print(zone_sales)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_3.py

import pandas as pd

def group_by_zone(df: pd.DataFrame, region_mapping: dict) -> pd.DataFrame:




#### Test 7.3 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 3

### 7.4 Grouping with Functions

Question:

Write a function that groups the sales data based (see problems above) on a custom grouping function, which groups the rows by their Quantity:

- If the Quantity is less than 5, it belongs to the group `Low`.

- If the Quantity is between 5 and 10, it belongs to the group `Medium`.

- If the Quantity is greater than 10, it belongs to the group `High`.

- Your task is to group the data based on this custom logic and calculate the total 

- SalesAmount for each of the custom quantity groups: "Low", "Medium", and "High".

Return as result a DataFrame with columns **Quantity**, **QuantityGroup**, and **SalesAmount**.

Hint:

- Use the `groupby()` function in Pandas, and pass a custom function `categorize_quantity()` to group by conditions based on the Quantity column.

- You can define a function `categorize_quantity` that categorizes the Quantity values and use it inside the `groupby()` function.

In [ ]:
import pandas as pd

def group_by_quantity_custom(df: pd.DataFrame) -> pd.DataFrame:
    
    # Write your code here
    



In [ ]:
#EXAMPLE USAGE
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

# Call the function
grouped_sales = group_by_quantity_custom(df)

print(grouped_sales)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_4.py
import pandas as pd

def group_by_quantity_custom(df: pd.DataFrame) -> pd.DataFrame:
    
    # Write your code here
    



#### Test 7.4 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 4

### 7.5 Grouping with Functions

Question:

Write a function that groups the sales data by Region, and for each region, apply a custom function to calculate the total revenue (SalesAmount) per Quantity. 

The custom function should calculate the **revenue per item sold** which is defined as **SalesAmount / Quantity** for each region.

Hint:
- Use the `groupby()` function to group the data by Region.
- Use the `.apply()` function to apply a custom function to each group.
- In the custom function, calculate the revenue per item for each region.

In [ ]:
import pandas as pd

def calculate_revenue_per_item(df: pd.DataFrame) -> pd.DataFrame:




In [ ]:
# EXAMPLE USAGE
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

grouped_sales = calculate_revenue_per_item(df)

print(grouped_sales)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_5.py

import pandas as pd

def calculate_revenue_per_item(df: pd.DataFrame) -> pd.DataFrame:



#### Test 7.5 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 5

### 7.6 Calculating Range of Sales per Region Using apply()

Problem: 

You are given the same CSV file **sample_sales_data_new.csv**. 
Write a function that groups the data by Region and calculates the range of SalesAmount for each region (i.e., the difference between the **maximum and minimum SalesAmount**).

Return a DataFrame with columns **Region** and **SalesRange**.

Hint:

- Use the `groupby()` function to group the data by Region.
- Use the `.apply()` function with a custom function to calculate the range (max - min) of SalesAmount for each region.


In [ ]:
import pandas as pd

def calculate_sales_range(df: pd.DataFrame) -> pd.DataFrame:
    
    # Write your code here
    



In [ ]:
##EXAMPLE USAGE

import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

# Call the function to calculate the range of sales
sales_range = calculate_sales_range(df)

# Print the result
print(sales_range)


#### Save your solution to a file 

In [ ]:
%%writefile solution_7_6.py

import pandas as pd

def calculate_sales_range(df: pd.DataFrame) -> pd.DataFrame:




#### Test 7.6 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 6

### 7.7 Column-Wise and Multiple Function Application on Multiple Columns

Question:

Write a function that groups the sales data by Region and applies multiple aggregation functions on the **Quantity** and **SalesAmount** columns. 

Return a DataFrame with columns **'Region'**, **'TotalQuantity'**, **'MeanQuantity'**, **'TotalSales'**, **'MaxSales'**.

Specifically:

- For the **Quantity** column, calculate the sum and mean.
- For the **SalesAmount column**, calculate the sum and max.

Hint:

- Use the `groupby()` function to group the data by Region.
- Use the `agg()` function to apply multiple aggregation functions to different columns.



In [ ]:
import pandas as pd

def aggregate_region_data(df: pd.DataFrame) -> pd.DataFrame:

    # Write your code here
    



In [ ]:
#EXAMPLE USAGE 
# Load the CSV file into a DataFrame
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

aggregated_sales = aggregate_region_data(df)

print(aggregated_sales)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_7.py

import pandas as pd

def aggregate_region_data(df: pd.DataFrame) -> pd.DataFrame:




#### Test 7.7 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 7

### 7.8 Apply: General split-apply-combine

Question:

Write a function that groups the data by Region and applies a user-defined function to calculate a custom score for each region 
based on both Quantity and SalesAmount. The custom score is calculated as:

`Score = (Total SalesAmount/Total Quantity) × (Number of Sales)`

Your task is to group the data by Region, apply this user-defined function, and return the score for each region

Return as result a DataFrame with columns **Region** and **Score**.

Hint:

- Use the `groupby()` function to group the data by Region.
- Define a custom function to calculate the score based on the aggregation of Quantity and SalesAmount.
- Use `apply()` to apply the user-defined function on each group.
- Avoid division by zero, i.e. if the value for **Total Quantity** is returned as zero, set the value for **Score** also to zero.

In [ ]:
import pandas as pd

def calculate_region_score(df: pd.DataFrame) -> pd.DataFrame:

    # Write your code here
    



In [ ]:
#EXAMPLE USAGE
# Load the CSV file into a DataFrame
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

region_scores = calculate_region_score(df)

print(region_scores)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_8.py

import pandas as pd

def calculate_region_score(df: pd.DataFrame) -> pd.DataFrame:




#### Test 7.8 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 8

### 7.9 Using the .transform() Method for Group-Wise Transformations

Question:

Write a function that groups the data by Region and uses the `.transform()` method to calculate the total sales for each region.

But instead of returning the aggregated value, assign the total sales back to each row in the group (i.e. 'unwrap' the aggregation results)

The final DataFrame should have the original columns with an additional column called **TotalSalesByRegion**, which will contain the total sales amount for the respective region.

 Hint:
 
- Use the `groupby()` function to group the data by Region.
- Use the `.transform()` method to calculate the total SalesAmount for each region.
- The `.transform()` method will 'broadcast' the total sales back to each row in the group.
                                                                                                            

In [ ]:
import pandas as pd

def add_total_sales_by_region(df: pd.DataFrame) -> pd.DataFrame:
    
    # Write your code here



In [ ]:
# EXAMPLE USAGE

df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

df_with_totals = add_total_sales_by_region(df)

print(df_with_totals)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_9.py

import pandas as pd

def add_total_sales_by_region(df: pd.DataFrame) -> pd.DataFrame:



#### Test 7.9 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 9

### 7.10 Creating Pivot Tables with the .pivot_table() Method

Question

Write a function that uses the `.pivot_table()` method to create a pivot table where:

- The rows represent **Region**.
- The columns represent **Product**.
- The values represent the sum of **SalesAmount**.
- Any missing values should be filled with 0.

Hint:

- Use the `.pivot_table()` method in Pandas.
- Use the index parameter to specify the rows and columns parameter to specify the columns of the pivot table.
- Use the argument **'aggfunc=sum'** to aggregate the SalesAmount and use fill_value=0 to handle missing values.


In [ ]:
import pandas as pd

def create_sales_pivot_table(df: pd.DataFrame) -> pd.DataFrame:

    # Write your code here
    



#### Example usage:

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/data/IFI8410/sess07/sample_sales_data_new.csv')

sales_pivot_table = create_sales_pivot_table(df)

print(sales_pivot_table)


#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_10.py

import pandas as pd

def create_sales_pivot_table(df: pd.DataFrame) -> pd.DataFrame:



#### Test 7.10 Execute the cell below to test your solution...


In [ ]:
! test/run_test.sh 10

## Regular Expressions and Text Analysis

### 7.11 Count word occurrences in a text

Problem:

You are given a paragraph and a target word. Implement the following function `count_word_occurrences_advanced`: 

This function takes a paragraph (string) and a target word (string) as input. 

It should:

- Find the number of occurrences of the target word in the paragraph using `re.findall()` with case insensitivity.

- Ensure that the target word matches as a whole word only (not part of other words).

- Allow for punctuation and special characters (like commas, periods, or exclamation marks) immediately following or preceding the target word without affecting the count.

Hint:

- Use word boundaries (\b) around the target word and use `re.IGNORECASE` for case insensitivity.

In [ ]:
import re

def count_word_occurrences_advanced(paragraph, word):
    
    # Write your code here



#### Example usage:

In [ ]:
# Example sentence and target word
sentence = "Python is great! python developers love Python programming."
target_word = "python"

# Apply the function
count = count_word_occurrences_advanced(sentence, target_word)
print(f"The word '{target_word}' appears {count} times in the sentence.")

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_11.py

import re

def count_word_occurrences_advanced(paragraph, word):



#### Test 7.11 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 11

### 7.12 Replace a substring in a sentence (longer string)

Question:

You are given a sentence and a target substring. Implement the following function `replace_substring`: 

This function takes three parameters as input arguments - a sentence (string), a target substring (string), and a replacement string. 

It should:

- Replace all occurrences of the target substring in the sentence with the replacement string, using `re.sub()`.

- Perform the replacement in a case-insensitive manner.

- Return the modified sentence.

Hint:

- Use `re.sub()` with the `re.IGNORECASE` flag to replace the target substring regardless of case.

In [ ]:
import re

def replace_substring(sentence, target, replacement):

    # Write your code here




#### Example usage:

In [ ]:
# Example sentence, target substring, and replacement string
sentence = "The cat sat on the Cat's mat and scared another cat."
target_substring = "cat"
replacement = "dog"

# Apply the function
modified_sentence = replace_substring(sentence, target_substring, replacement)
print(modified_sentence)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_12.py

import re

def replace_substring(sentence, target, replacement):




#### Test 7.12 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 12

### 7.13 Find words of a specific length in a text

Problem:

You are given a sentence and a target word length. Implement the following function `find_words_with_length`: 

This function takes two parameters - a sentence (string) and a target length (integer). 

It should:

- Find all words in the sentence that have the exact number of characters specified by the target length.
  
- Return the list of these words.

Hint:

- Use `re.findall()` to match words of the given length, and ensure it excludes punctuation marks.

In [ ]:
import re

def find_words_with_length(sentence, length):
    
    # Write your code here



#### Example usage:

In [ ]:
# Example sentence and target length
sentence = "Python is amazing and fun! I enjoy coding in Python every day."
target_length = 5

# Apply the function
words_with_length = find_words_with_length(sentence, target_length)
print(words_with_length)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_13.py

import re

def find_words_with_length(sentence, length):



#### Test 7.13 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 13

### 7.14 Find words in a text start and end with specific letters

Problem:

You are given a sentence and two letters - a starting letter and an ending letter. Implement the following function `find_words_start_end()`: 

This function takes three parameters - a sentence (string), a start letter (string), and an end letter (string). 

It should:

- Find all words in the sentence that start with a specified start letter and end with a specified end letter.

- Perform the search in a case-insensitive manner.

- Return the list of words that match the criteria.

Hint:

- Use `re.findall()` with a regex pattern that includes the start and end letters as variables.

In [ ]:
import re

def find_words_start_end(sentence, start_letter, end_letter):
    
    # Write your code here



#### Example usage:

In [ ]:
# Example sentence, start letter, and end letter
sentence = "Snakes slither silently in the shadows and swiftly strike."
start_letter = "s"
end_letter = "y"

# Apply the function
matching_words = find_words_start_end(sentence, start_letter, end_letter)
print(matching_words)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_14.py

import re

def find_words_start_end(sentence, start_letter, end_letter):



#### Test 7.14 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 14

### 7.15 Transform a text file

Problem:

You are given a text file (**RE_Text.txt**) containing a paragraph about regular expressions. 

Implement the following function `transform_text()`:

This function takes the file path (string) as input, reads the text from the file, and applies the following transformations:

- Replace all occurrences of the substring "regex" with "regular expressions".

- Find and return words of a specific length, storing them in a list.

- Find and return words that start with one letter and end with another, storing them in another list.

The function should return the modified text (as a string) and the two lists as a tuple.

Additionally, the function should take extra parameters for:

- The word length for finding specific-length words.

- The start and end letters for the letter-based search.

In [ ]:
import re

def transform_text(file_path, word_length, start_letter, end_letter):
  
    # Write your code here



#### Example usage:

In [ ]:
# Example usage after modifying the function
file_path = "/data/IFI8410/sess07/RE_Text.txt"
word_length = 5
start_letter = "r"
end_letter = "s"

# Apply the function
modified_text, length_words, start_end_words = transform_text(file_path, word_length, start_letter, end_letter)

# Display results
print("Modified Text:", modified_text)
print("Words of specific length:", length_words)
print("Words that start with 'r' and end with 's':", start_end_words)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_15.py

import re

def transform_text(file_path, word_length, start_letter, end_letter):



#### Test 7.15 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 15

### 7.16 Find the first word in a textr that follows a specific pattern

Problem:

Define a function `find_word_with_pattern()` that uses `re.search()` to find the first occurrence of a word in a sentence that starts with a specific letter and ends with another letter.

This function takes the following parameters:

- A sentence (string)

- A start_letter (string)

- An end_letter (string)

It should return the first word that starts with the specified **start_letter** and ends with the specified **end_letter**. If no such word is found, it should return None.

Hint:

- Use `re.findall()` to find all matching words.
- Dynamically generate the regex pattern using the start and end letters.
- If multiple matches are found, return the longest match using the max() function.

In [ ]:
import re

def find_word_with_pattern(sentence, start_letter, end_letter):
    
    # Write your code here
    


#### Example usage:

In [ ]:
# Example usage:
sentence = "Rabbits are running fast and reach their goals."
start_letter = "r"
end_letter = "s"

result = find_word_with_pattern(sentence, start_letter, end_letter)

In [ ]:
# Example usage block for students:

# Example 1: Simple case where the word starts with 'r' and ends with 's'
sentence = "Rabbits are running fast and reach their goals."
start_letter = "r"
end_letter = "s"
print(find_word_with_pattern(sentence, start_letter, end_letter))  # Expected output: "Rabbits"

# Example 2: Case where multiple words match but only the longest should be returned
sentence = "I love programming in python."
start_letter = "p"
end_letter = "g"
print(find_word_with_pattern(sentence, start_letter, end_letter))  # Expected output: "programming"

# Example 3: Case where no match is found
sentence = "There are no such words here."
start_letter = "x"
end_letter = "z"
print(find_word_with_pattern(sentence, start_letter, end_letter))  # Expected output: None

# Example 4: Case insensitivity check
sentence = "Red roses bloom every spring."
start_letter = "r"
end_letter = "s"
print(find_word_with_pattern(sentence, start_letter, end_letter))  # Expected output: "roses"

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_16.py

import re

def find_word_with_pattern(sentence, start_letter, end_letter):



#### Test 7.16 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 16

### 7.17 Check whether a sentence starts with a specific word

Problem:

Use `re.match()` to check if a sentence starts with a certain word.

The user-defined function `check_start_word()` shall take the following input variables:

- A sentence (string)

- A word (string)

It should return **True** if the sentence starts with the given word, otherwise return **False**.

Hint:

- Use `re.match()` to check if the word appears at the start of the sentence.

- Create a regex pattern that matches the word at the beginning of the sentence using the ^ symbol.


In [ ]:
import re

def check_start_word(sentence, word):
    
    # Write your code here




#### Example usage:

In [ ]:
# Example usage:
sentence = "Python is a great language."
word = "Python"

result = check_start_word(sentence, word)

In [ ]:
# Test case 1
sentence = "Python is a great language."
word = "Python"
result = check_start_word(sentence, word)
print(result)  # Expected Output: True

# Test case 2
sentence = "Java is widely used."
word = "Python"
result = check_start_word(sentence, word)
print(result)  # Expected Output: False

# Test case 3
sentence = "JavaScript frameworks are popular."
word = "javascript"
result = check_start_word(sentence, word)
print(result)  # Expected Output: True (case-insensitive)

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_17.py

import re

def check_start_word(sentence, word):



#### Test 7.17 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 17

### 7.18 Extract emails from a text

Problem:

Use `re.compile()` to compile a regular expression pattern that matches email addresses.

The function `extract_emails()` shall take the following parameter:

- A list of strings (containing potential email addresses)

- It should return a list of all valid email addresses using the compiled regular expression pattern.

Hint:

- Use `re.compile()` to compile a regex pattern for matching email addresses.

- Iterate through the list of strings and apply `findall()` to extract all matches.

- Ensure the pattern allows for common email formats, including letters, numbers, dots, and hyphens.

- Consult the recent lecture notebooks in class to construct the regex pattern.

In [ ]:
import re

def extract_emails(strings):
    
    # Write your code here
    


#### Example usage:

In [ ]:
# Example usage:
strings = [
    "Contact us at support@example.com",
    "This is not an email address",
    "Send your queries to info@mydomain.org or feedback@company.net",
    "random_text_with_no_email"
]

result = extract_emails(strings)
print(result)  # Output: ['support@example.com', 'info@mydomain.org', 'feedback@company.net']

In [ ]:
# Test case 1
strings = [
    "Contact us at support@example.com",
    "This is not an email address",
    "Send your queries to info@mydomain.org or feedback@company.net",
    "random_text_with_no_email"
]
result = extract_emails(strings)
print(result)  # Expected Output: ['support@example.com', 'info@mydomain.org', 'feedback@company.net']

# Test case 2
strings = ["noemails@here", "randomstring", "valid_email@domain.com"]
result = extract_emails(strings)
print(result)  # Expected Output: ['valid_email@domain.com']

# Test case 3
strings = ["not_an_email@domain", "test@domain.co.uk"]
result = extract_emails(strings)
print(result)  # Expected Output: ['test@domain.co.uk']

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_18.py

import re

def extract_emails(strings):




#### Test 7.18 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 18

### 7.19 Match strings that begin with a specific sequence of characters followed by any two digits

Problem:

Write a function `match_sequence_at_start` that matches strings that begin with a specific sequence of characters followed by any two digits.

This function takes the following parameters:

- A list of strings (strings to be matched)

- A start_sequence (specific sequence of characters)

It should return a list of strings that match the pattern, where the pattern starts with the given sequence and is followed by exactly two digits (\d\d).

Hint:

- Use `re.compile()` to compile a regex pattern starting with the given sequence followed by two digits.

- Use `^` to anchor the match to the start of the string.

- Iterate through the list and use `match()` to check each string against the compiled pattern.

In [ ]:
import re

def match_sequence_at_start(strings, start_sequence):
    
    # Write your code here




#### Example usage:

In [ ]:
# Example usage:
strings = ["AB12", "XY34", "AB56", "A123", "AB78", "XY90"]
start_sequence = "AB"

result = match_sequence_at_start(strings, start_sequence)

In [ ]:
# Test case 1
strings = ["AB12", "XY34", "AB56", "A123", "AB78", "XY90"]
start_sequence = "AB"
result = match_sequence_at_start(strings, start_sequence)
print(result)  # Expected Output: ['AB12', 'AB56', 'AB78']

# Test case 2
strings = ["XY12", "XY34", "XY56", "AB78", "XY90"]
start_sequence = "XY"
result = match_sequence_at_start(strings, start_sequence)
print(result)  # Expected Output: ['XY12', 'XY34', 'XY56', 'XY90']

# Test case 3
strings = ["AB12", "AB1234", "XYZ78", "XY45"]
start_sequence = "AB"
result = match_sequence_at_start(strings, start_sequence)
print(result)  # Expected Output: ['AB12']

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_19.py

import re

def match_sequence_at_start(strings, start_sequence):



#### Test 7.19 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 19

### 7.20 Match strings that end with a specific sequence of characters followed by any two digits


Problem:

Write a function `match_sequence_at_end` that matches strings that end with a specific sequence of characters followed by any two digits.

This function takes the following parameters:

- A list of strings (strings to be matched)

- An end_sequence (specific sequence of characters)

It should return a list of strings that match the pattern, where the pattern ends with the given sequence and is preceded by exactly two digits (\d\d).

Hint:

- Use `re.compile()` to create a regex pattern that ends with the given sequence, preceded by two digits.

- Use `$` to anchor the match to the end of the string.

- Iterate through the list and use search() to check each string against the compiled pattern.

In [ ]:
import re

def match_sequence_at_end(strings, end_sequence):
    
    # Write your code here



#### Example usage:

In [ ]:
# Example usage:
strings = ["12AB", "34XY", "56AB", "123A", "78AB", "90XY"]
end_sequence = "AB"

result = match_sequence_at_end(strings, end_sequence)

In [ ]:
# Test case 1
strings = ["12AB", "34XY", "56AB", "123A", "78AB", "90XY"]
end_sequence = "AB"
result = match_sequence_at_end(strings, end_sequence)
print(result)  # Expected Output: ['12AB', '56AB', '78AB']

# Test case 2
strings = ["12XY", "34XY", "56XY", "12AB", "78XY", "90XY"]
end_sequence = "XY"
result = match_sequence_at_end(strings, end_sequence)
print(result)  # Expected Output: ['12XY', '34XY', '56XY', '78XY', '90XY']

# Test case 3
strings = ["AB12", "12XY34", "XY56AB", "AB78"]
end_sequence = "XY56"
result = match_sequence_at_end(strings, end_sequence)
print(result)  # Expected Output: ['56AB']

#### Save your solution to a file ...

In [ ]:
%%writefile solution_7_20.py

import re

def match_sequence_at_end(strings, end_sequence):



#### Test 7.20 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 20

# Run all the tests again ...

In [ ]:
! ./test/run_test.sh

# Homework Submission
- This homework is due by **2024-10-30, 6:00 PM (EDT)**.
- Make sure that all your programs and output files are in the exact folder as specified in the instructions.
- All file names on this system are case sensitive. Verify if you copy your work from a local computer to your home directory on ARC.
- **Execute the cell below to submit your assignment**

In [ ]:
! ./submit.sh -y